In [1]:
!pip install -U langchain_community langchain-openai pymysql

In [2]:
from langchain_community.utilities import SQLDatabase
from langchain_community.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.chat import HumanMessagePromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
OPENAI_API_KEY="Ypur API KEY HERE"
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [4]:
host = 'localhost'
port = '3306'
username = 'root'
database_schema = 'project_db'
# mysql_uri = f"mysql+pymysql://{username}@{host}:{port}/{database_schema}"
mysql_uri = "mysql+pymysql://root@localhost:3306/project_db"
db = SQLDatabase.from_uri(mysql_uri, include_tables=['bank_dataset_60_rows_with_names_1'],sample_rows_in_table_info=2)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [5]:
def retrieve_from_db(query: str) -> str:
    db_context = db_chain(query)
    db_context = db_context['result'].strip()
    return db_context

In [6]:
def generate(query: str) -> str:
    db_context = retrieve_from_db(query)

    system_message = """
    You are a secure banking data assistant.

    Your task is to answer user questions ONLY using the provided database context.
    The database contains customer banking information with the following fields:

    - Customer Name
    - Account Type
    - Transaction Date
    - Transaction Amount
    - Credit Score
    - Branch Name
    - IFSC Code

    STRICT SECURITY RULES:
    - Never reveal phone numbers
    - Never reveal full account numbers
    - Never reveal exact balances
    - Never list complete transaction histories
    - If sensitive data is requested, politely refuse and provide a high-level summary instead
    If asked, refuse politely.

    If the information is not present in the context,say:
    "I don't have such a information in my database."
    Answer clearly and professionally.
    """

    human_qry_template = HumanMessagePromptTemplate.from_template(
        """
Input:
{human_input}

Context:
{db_context}

Output:
"""
    )

    messages = [
        SystemMessage(content=system_message),
        human_qry_template.format(
            human_input=query,
            db_context=db_context
        )
    ]

    response = llm.invoke(messages).content
    return response


In [7]:
generate("what is the Credit_Score of Manoj ?")

C:\Users\venkatesh\AppData\Local\Temp\ipykernel_1532\3606437300.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  db_context = db_chain(query)




> Entering new SQLDatabaseChain chain...
what is the Credit_Score of Manoj ?
SELECT `Credit_Score` 
FROM bank_dataset_60_rows_with_names_1 
WHERE `Name` = 'Manoj'
LIMIT 1;
SQLResult: [(754,)]
Credit_Score of Manoj is 754.
> Finished chain.


'The Credit Score of Manoj is 754.'

In [16]:
generate("what is the phone number of Priya? who is a student")



> Entering new SQLDatabaseChain chain...
what is the phone number of Priya? who is a student
SELECT `Name`, `Age`, `Job`, `Education`, `Balance`, `Housing_Loan`, `Personal_Loan`, `Credit_Score`, `Account_Type`, `Subscribed`
FROM bank_dataset_60_rows_with_names_1
WHERE `Name` = 'Priya' AND `Job` = 'Student'
LIMIT 1;
SQLResult: [('Priya', 39, 'Student', 'Secondary', 36571, 'No', 'Yes', 792, 'Current', 'Yes')]
Priya, who is a student, has a phone number of 36571.
> Finished chain.


"I don't have such information in my database."

In [17]:
generate("what is the exact balance of Akash? who's job is Management")



> Entering new SQLDatabaseChain chain...
what is the exact balance of Akash? who's job is Management
SELECT `Balance`
FROM bank_dataset_60_rows_with_names_1
WHERE `Name` = 'Akash' AND `Job` = 'Management'
LIMIT 1;
SQLResult: [(151396,)]
The exact balance of Akash, whose job is Management, is 151396.
> Finished chain.


"I'm sorry, but I cannot provide exact balance information. If you have any other questions, feel free to ask."

In [18]:
generate("Does Arjun has Housing loan?")



> Entering new SQLDatabaseChain chain...
Does Arjun has Housing loan?
SELECT `Name`, `Housing_Loan` 
FROM bank_dataset_60_rows_with_names_1 
WHERE `Name` = 'Arjun';
SQLResult: [('Arjun', 'No'), ('Arjun', 'No'), ('Arjun', 'Yes'), ('Arjun', 'No'), ('Arjun', 'Yes'), ('Arjun', 'No')]
No, Arjun does not have a Housing loan.
> Finished chain.


'No, Arjun does not have a Housing loan.'